In [ ]:
!pip -q install ultralytics==8.3.50 opencv-python-headless==4.10.0.84 albumentations==1.4.7 kaggle roboflow==1.1.47
import os, sys, json, shutil, math, zipfile, glob, re, yaml
from pathlib import Path
print("✓ deps installed")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.0 MB/s eta 0:00:00
✓ deps installed


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

DRIVE_ROOT = Path("/content/drive/MyDrive/indrive_project").resolve()
DATA_DIR   = DRIVE_ROOT/"datasets"     # сюда сложим датасеты
RUNS_DIR   = DRIVE_ROOT/"runs"         # сюда Ultralytics будет писать тренировки
BUNDLE_DIR = DRIVE_ROOT/"bundles"      # сюда будем класть zip bundle

for p in [DATA_DIR, RUNS_DIR, BUNDLE_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Drive root:", DRIVE_ROOT)


Mounted at /content/drive
Drive root: /content/drive/MyDrive/indrive_project


In [ ]:
# пусть все датасеты и runs живут в Google Drive (персистентно)
!yolo settings datasets_dir="{DATA_DIR}" runs_dir="{RUNS_DIR}"
!yolo settings | sed -n '1,80p'


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/drive/MyDrive/indrive_project/datasets",
  "weights_dir": "weights",
  "runs_dir": "/content/drive/MyDrive/indrive_project/runs",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
JSONDict("/root/.co

In [ ]:
from google.colab import files
files.upload()  # выберите kaggle.json
!mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Roboflow: https://roboflow.com/ -> API Key
# Kaggle: kaggle.json (Account -> Create API Token) в /root/.kaggle/kaggle.json
import os, json, pathlib

os.environ["ROBOFLOW_API_KEY"] = "wQibjhV0PTXn67h2O3Xd"

# Если есть kaggle.json в 'files' — скопируйте:
from pathlib import Path
if Path("/content/kaggle.json").exists():
    Path("~/.kaggle").expanduser().mkdir(exist_ok=True)
    shutil.copy("/content/kaggle.json", str(Path("~/.kaggle/kaggle.json").expanduser()))
    os.chmod(str(Path("~/.kaggle/kaggle.json").expanduser()), 0o600)

print("RF key set:", bool(os.environ.get("ROBOFLOW_API_KEY")))
print("Kaggle exists:", Path("~/.kaggle/kaggle.json").expanduser().exists())


RF key set: True
Kaggle exists: True


In [ ]:
from roboflow import Roboflow

TARGET = DATA_DIR/"damage"  # итоговый датасет детекции
for p in [TARGET/"images/train", TARGET/"images/val", TARGET/"labels/train", TARGET/"labels/val"]:
    p.mkdir(parents=True, exist_ok=True)

def norm(s):
    s=(s or "").lower().strip()
    s=re.sub(r"[:\-\/\\\.,]"," ",s)
    return "_".join(s.split())

ALIAS = {
    # целевые классы
    "scratch":"scratch","scratches":"scratch","scracth":"scratch","scrach":"scratch",
    "car_scratch":"scratch","car-scratch":"scratch","damaged_scratch":"scratch",
    "dent":"dent","dunt":"dent","severe_damage":"dent","severe":"dent","damaged_dent":"dent",
    "rust":"rust",
    # игнор
    "good_condition":None,"car":None,"dirt":None,"0":None
}
def map_name(cname):
    n=norm(cname)
    if n in ALIAS: return ALIAS[n]
    if "scratch" in n: return "scratch"
    if "rust" in n:    return "rust"
    if "dent" in n or "severe" in n: return "dent"
    return None  # игнор

IMG_EXT={".jpg",".jpeg",".png"}

def parse_line(toks):
    # YOLO: c xc yc w h | (полигон) c x1 y1 x2 y2 ...
    coords=[]
    for t in toks[1:]:
        try:v=float(t)
        except:continue
        if 0.0<=v<=1.0: coords.append(v)
    if len(coords)==4:
        return ("bbox",tuple(coords))
    if len(coords)>=6 and len(coords)%2==0:
        xs=[coords[i] for i in range(0,len(coords),2)]
        ys=[coords[i+1] for i in range(0,len(coords),2)]
        x1,x2=min(xs,default=0),max(xs,default=0)
        y1,y2=min(ys,default=0),max(ys,default=0)
        return ("bbox",((x1+x2)/2,(y1+y2)/2,max(0,x2-x1),max(0,y2-y1)))
    return None

def is_full(xc,yc,w,h):
    # отбрасываем "во весь кадр" фейковую разметку
    return (w>=0.995) or (h>=0.995) or (w*h>=0.98)

def resolve_imglbl(src_dir: Path, rel: str, hint: str):
    p=(src_dir/rel).resolve()
    if p.exists(): return p, p.parent/"labels"
    cand = src_dir/hint/"images"; lbl = src_dir/hint/"labels"
    if cand.exists() and lbl.exists(): return cand,lbl
    for s in ["train","valid","val","test"]:
        cand=src_dir/s/"images"; lbl=src_dir/s/"labels"
        if cand.exists() and lbl.exists(): return cand,lbl
    return None,None

def merge_dir(src_dir: Path):
    stats={"kept":0,"badcls":0,"nolabel":0,"full":0}
    if not (src_dir/"data.yaml").exists():
        print(f"skip {src_dir}: no data.yaml")
        return stats
    data=yaml.safe_load(open(src_dir/"data.yaml"))
    names=data["names"]
    id2name={i:v for i,v in enumerate(names)} if isinstance(names,list) else {int(k):v for k,v in names.items()}

    yaml_splits={}
    for k in ["train","val","valid","test"]:
        if data.get(k): yaml_splits[k]=data[k]

    for k,rel in yaml_splits.items():
        out_split = "train" if k=="train" else "val"
        img_dir,lbl_dir = resolve_imglbl(src_dir, rel, "valid" if k=="val" else k)
        if not img_dir or not lbl_dir:
            print(f"  ! can't resolve {k}: {rel}");
            continue

        for img in img_dir.rglob("*"):
            if img.suffix.lower() not in IMG_EXT:
                continue
            lbl = (lbl_dir / img.relative_to(img_dir)).with_suffix(".txt")
            if not lbl.exists(): stats["nolabel"]+=1; continue

            rows=[]
            for line in open(lbl):
                toks=line.strip().split()
                if not toks: continue
                try: cid=int(float(toks[0]))
                except: continue
                cname=id2name.get(cid,"")
                t=map_name(cname)
                if t is None: stats["badcls"]+=1; continue
                parsed=parse_line(toks)
                if not parsed: continue
                _,(xc,yc,w,h)=parsed
                if is_full(xc,yc,w,h): stats["full"]+=1; continue
                tid={"scratch":0,"dent":1,"rust":2}[t]
                rows.append((tid,xc,yc,w,h))
            if not rows: continue

            if out_split=="train":
                out_img = TARGET/"images/train"/img.name
                out_lbl = TARGET/"labels/train"/(img.stem+".txt")
            else:
                out_img = TARGET/"images/val"/img.name
                out_lbl = TARGET/"labels/val"/(img.stem+".txt")
            out_img.parent.mkdir(parents=True, exist_ok=True)
            out_lbl.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(img,out_img)
            with open(out_lbl,"w") as f:
                for tid,xc,yc,w,h in rows:
                    f.write(f"{tid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")
            stats["kept"]+=1
    return stats

# === 4.1 Скачать Roboflow наборы (если ещё не скачаны) ===
assert os.environ.get("ROBOFLOW_API_KEY"), "Set ROBOFLOW_API_KEY in the previous cell!"
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

def dl_project(workspace, project):
    p = rf.workspace(workspace).project(project)
    v = p.versions()[0]  # latest
    ds = v.download("yolov8")
    print("✓ Downloaded:", ds.location)
    return Path(ds.location).resolve()

rf_dirs = []
rf_dirs.append(dl_project("milo", "car-damages-abjy0"))        # names: damaged- dent, damaged- scratch, ...
rf_dirs.append(dl_project("seva-at1qy", "rust-and-scrach"))    # names: car, dunt, rust, scracth

# === 4.2 Слить в единый датасет ===
report = {"kept":0,"badcls":0,"nolabel":0,"full":0}
for d in rf_dirs:
    s = merge_dir(d)
    print(f"• {d.name} -> {s}")
    for k in report: report[k]+=s[k]

# data_damage.yaml с абсолютным путём (!)
data_damage_yaml = {
    "path": str(TARGET.resolve()),
    "train": "images/train",
    "val": "images/val",
    "names": ["scratch","dent","rust"]
}
with open("data_damage.yaml","w") as f:
    yaml.safe_dump(data_damage_yaml, f, sort_keys=False, allow_unicode=True)

print("REPORT:", json.dumps(report, indent=2))
print(open("data_damage.yaml").read())

print("Counts:",
      len(list((TARGET/'images/train').glob('*'))),"train imgs,",
      len(list((TARGET/'images/val').glob('*'))),"val imgs")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Car-damages-13 in yolov8:: 100%|██████████| 76548/76548 [00:12<00:00, 6277.49it/s]


✓ Downloaded: /content/Car-damages-13
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Rust-and-Scrach-1 in yolov8:: 100%|██████████| 218/218 [00:00<00:00, 6687.13it/s]


✓ Downloaded: /content/Rust-and-Scrach-1
• Car-damages-13 -> {'kept': 38, 'badcls': 3384, 'nolabel': 0, 'full': 34807}
• Rust-and-Scrach-1 -> {'kept': 61, 'badcls': 71, 'nolabel': 0, 'full': 0}
REPORT: {
  "kept": 99,
  "badcls": 3455,
  "nolabel": 0,
  "full": 34807
}
path: /content/drive/MyDrive/indrive_project/datasets/damage
train: images/train
val: images/val
names:
- scratch
- dent
- rust

Counts: 83 train imgs, 16 val imgs


In [ ]:
# === Робастная сборка Kaggle severity (minor/moderate/major) ===
from pathlib import Path
import shutil, os

DRIVE_ROOT = Path("/content/drive/MyDrive/indrive_project").resolve()
DATA_DIR   = DRIVE_ROOT/"datasets"
KAG_DST    = DATA_DIR/"severity_raw"
KAG_OUT    = DATA_DIR/"severity"
(KAG_OUT/"train").mkdir(parents=True, exist_ok=True)
(KAG_OUT/"val").mkdir(parents=True, exist_ok=True)

# 1) найдём корень с двумя подпапками training/validation внутри датасета
root = None
candidate_root = KAG_DST/"car-damage-severity-dataset"

def find_tv(base: Path):
    """Ищем пары (training, validation) на любых уровнях вложенности."""
    trainings = list(base.rglob("training"))
    validations = list(base.rglob("validation"))
    pairs = []
    for t in trainings:
        # ищем в том же родителе папку validation
        v = t.parent/"validation"
        if v.exists():
            pairs.append((t, v))
    # fallback: если нашлись в разных местах — выбираем ближайшие по глубине
    if not pairs and trainings and validations:
        # берём первую пару одинакового уровня
        for t in trainings:
            for v in validations:
                if t.parent == v.parent:
                    pairs.append((t, v))
                    break
            if pairs: break
    return pairs[0] if pairs else (None, None)

if candidate_root.exists():
    t, v = find_tv(candidate_root)
    if t and v:
        root = candidate_root
        train_src, val_src = t, v
else:
    # бывает другой корень — поищем по всему KAG_DST
    t, v = find_tv(KAG_DST)
    if t and v:
        root = t.parent
        train_src, val_src = t, v

if not root:
    # Диагностика: покажем дерево верхнего уровня
    print("Could not find training/validation. Top-level entries:")
    for p in sorted(KAG_DST.glob("*")):
        print(" -", p)
    raise FileNotFoundError("Не удалось найти папки 'training' и 'validation' внутри архива Kaggle.")

print("Detected root:", root)
print("Train src:", train_src)
print("Val src:", val_src)

# 2) Копируем в нужную структуру (train/val/{minor,moderate,major})
def copytree_flat(src, dst):
    dst.mkdir(parents=True, exist_ok=True)
    n = 0
    for p in src.rglob("*"):
        if p.is_file() and p.suffix.lower() in {".jpg",".jpeg",".png"}:
            shutil.copy2(p, dst/p.name)
            n += 1
    return n

mapping = {
    "01-minor":"minor",
    "minor":"minor",
    "02-moderate":"moderate",
    "moderate":"moderate",
    "03-severe":"major",
    "severe":"major"
}

def class_dir_candidates(base: Path):
    # собираем все подпапки с картинками; фильтруем по известным именам
    out = {}
    for sub in base.iterdir():
        if sub.is_dir():
            name = sub.name.lower().strip()
            if name in mapping or name in {"01-minor","02-moderate","03-severe"}:
                out[name] = sub
    return out

train_map = class_dir_candidates(train_src)
val_map   = class_dir_candidates(val_src)

assert train_map and val_map, f"Не нашли класс-папки в {train_src} / {val_src}"

counts = {"train":{}, "val":{}}
for k, v in train_map.items():
    tgt = mapping.get(k, k)
    counts["train"][tgt] = copytree_flat(v, KAG_OUT/"train"/tgt)
for k, v in val_map.items():
    tgt = mapping.get(k, k)
    counts["val"][tgt] = copytree_flat(v, KAG_OUT/"val"/tgt)

print("✓ severity train counts:", counts["train"])
print("✓ severity val counts:", counts["val"])

# sanity-print: сколько всего
def count_imgs(p):
    return sum(1 for _ in Path(p).rglob("*") if _.suffix.lower() in {".jpg",".jpeg",".png"})
print("TOTAL train:", count_imgs(KAG_OUT/"train"), "TOTAL val:", count_imgs(KAG_OUT/"val"))


Detected root: /content/drive/MyDrive/indrive_project/datasets/severity_raw/data3a
Train src: /content/drive/MyDrive/indrive_project/datasets/severity_raw/data3a/training
Val src: /content/drive/MyDrive/indrive_project/datasets/severity_raw/data3a/validation
✓ severity train counts: {'minor': 452, 'moderate': 463, 'major': 468}
✓ severity val counts: {'minor': 82, 'moderate': 75, 'major': 91}
TOTAL train: 1383 TOTAL val: 248


In [ ]:
from ultralytics import YOLO

# --- Детектор ---
det = YOLO("yolov8s.pt")
det_res = det.train(
    data="data_damage.yaml",
    imgsz=640, epochs=120, batch=-1, amp=True, workers=2,
    project=str(RUNS_DIR), name="v8s_det_damage", verbose=True
)
DET_PT = Path(det_res.save_dir)/"weights/best.pt"
print("Detector:", DET_PT)

# --- Классификатор серьёзности ---
cls = YOLO("yolov8s-cls.pt")
cls_res = cls.train(
    data=str((DATA_DIR/"severity").resolve()),  # абсолютный путь
    imgsz=256, epochs=50, batch=-1, amp=True, workers=2,
    project=str(RUNS_DIR), name="v8s_cls_severity", verbose=True
)
CLS_PT = Path(cls_res.save_dir)/"weights/best.pt"
print("Classifier:", CLS_PT)


100%|██████████| 21.5M/21.5M [00:00<00:00, 101MB/s] 


New https://pypi.org/project/ultralytics/8.3.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data_damage.yaml, epochs=120, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=/content/drive/MyDrive/indrive_project/runs, name=v8s_det_damage, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 95.0MB/s]


...

      50/50      0.96G     0.0498          7        256: 100%|██████████| 87/87 [00:15<00:00,  5.80it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:01<00:00,  6.62it/s]

                   all      0.673          1



50 epochs completed in 0.249 hours.
Optimizer stripped from /content/drive/MyDrive/indrive_project/runs/v8s_cls_severity/weights/last.pt, 10.3MB
Optimizer stripped from /content/drive/MyDrive/indrive_project/runs/v8s_cls_severity/weights/best.pt, 10.3MB

Validating /content/drive/MyDrive/indrive_project/runs/v8s_cls_severity/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-cls summary (fused): 73 layers, 5,079,043 parameters, 0 gradients, 12.5 GFLOPs
train: /content/drive/MyDrive/indrive_project/datasets/severity/train... found 1383 images in 3 classes ✅ 
val: /content/drive/MyDrive/indrive_project/datasets/severity/val... found 248 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 8/8 [00:01<00:00,  7.15it/s]


                   all      0.742          1
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/indrive_project/runs/v8s_cls_severity
Classifier: /content/drive/MyDrive/indrive_project/runs/v8s_cls_severity/weights/best.pt


In [ ]:
#retrain
from pathlib import Path
import os, re, yaml, shutil, hashlib, json

DRIVE_ROOT = Path("/content/drive/MyDrive/indrive_project").resolve()
DATA_DIR   = DRIVE_ROOT/"datasets"
RUNS_DIR   = DRIVE_ROOT/"runs"

TARGET = DATA_DIR/"damage"  # итоговый датасет детекции (мы его перезаполним train, val не трогаем)
(TARGET/"images/train").mkdir(parents=True, exist_ok=True)
(TARGET/"images/val").mkdir(parents=True, exist_ok=True)
(TARGET/"labels/train").mkdir(parents=True, exist_ok=True)
(TARGET/"labels/val").mkdir(parents=True, exist_ok=True)

IMG_EXT={".jpg",".jpeg",".png"}


In [ ]:
!pip -q install roboflow==1.1.47 ultralytics==8.3.50
from roboflow import Roboflow

assert os.environ.get("ROBOFLOW_API_KEY"), "Set ROBOFLOW_API_KEY in env"

rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

def dl(workspace, project):
    v = rf.workspace(workspace).project(project).versions()[0]
    ds = v.download("yolov8")
    print("✓", ds.location)
    return Path(ds.location).resolve()

rf_dirs = []
rf_dirs.append(dl("milo", "car-damages-abjy0"))        # уже был
rf_dirs.append(dl("seva-at1qy", "rust-and-scrach"))    # уже был
rf_dirs.append(dl("carpro", "car-scratch-and-dent"))   # НОВЫЙ
rf_dirs.append(dl("project-kmnth", "car-scratch-xgxzs")) # НОВЫЙ

rf_dirs


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`
✓ /content/Car-damages-13
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`
✓ /content/Rust-and-Scrach-1
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Car-Scratch-and-Dent-5 in yolov8:: 100%|██████████| 1262/1262 [00:00<00:00, 4938.54it/s]

✓ /content/Car-Scratch-and-Dent-5
loading Roboflow workspace...


loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.50, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to car--scratch-2 in yolov8:: 100%|██████████| 2454/2454 [00:00<00:00, 6077.71it/s]

✓ /content/car--scratch-2


[PosixPath('/content/Car-damages-13'),
 PosixPath('/content/Rust-and-Scrach-1'),
 PosixPath('/content/Car-Scratch-and-Dent-5'),
 PosixPath('/content/car--scratch-2')]

In [ ]:
def norm(s):
    s=(s or "").lower().strip()
    s=re.sub(r"[:\-\/\\\.,]"," ",s)
    return "_".join(s.split())

ALIAS = {
    "scratch":"scratch","scrach":"scratch","scratches":"scratch","scracth":"scratch","car_scratch":"scratch","car-scratch":"scratch","damaged_scratch":"scratch",
    "dent":"dent","dunt":"dent","severe_damage":"dent","severe":"dent","damaged_dent":"dent",
    "rust":"rust",
    "good_condition":None,"car":None,"dirt":None,"0":None
}
def map_name(cname):
    n=norm(cname)
    if n in ALIAS: return ALIAS[n]
    if "scratch" in n: return "scratch"
    if "rust"    in n: return "rust"
    if "dent"    in n or "severe" in n: return "dent"
    return None

def parse_line(line):
    toks=line.strip().split()
    if not toks: return None
    try: cid=int(float(toks[0]))
    except: return None
    coords=[]
    for t in toks[1:]:
        try:v=float(t)
        except:continue
        if 0<=v<=1: coords.append(v)
    if len(coords)==4:  # bbox
        return cid, tuple(coords)
    if len(coords)>=6 and len(coords)%2==0:  # polygon -> bbox
        xs=[coords[i] for i in range(0,len(coords),2)]
        ys=[coords[i+1] for i in range(0,len(coords),2)]
        x1,x2=min(xs),max(xs); y1,y2=min(ys),max(ys)
        xc,yc = (x1+x2)/2, (y1+y2)/2
        w,h   = max(0,x2-x1), max(0,y2-y1)
        return cid, (xc,yc,w,h)
    return None

def is_full(xc,yc,w,h):
    return (w>=0.995) or (h>=0.995) or (w*h>=0.98)

def is_tiny(w,h,thr=0.004):  # слишком маленький бокс (на 640×640 ~ 1.6к пикс.)
    return (w*h)<thr

def resolve_imglbl(src_dir: Path, rel: str, hint: str):
    p=(src_dir/rel).resolve()
    if p.exists(): return p, p.parent/"labels"
    cand = src_dir/hint/"images"; lbl = src_dir/hint/"labels"
    if cand.exists() and lbl.exists(): return cand,lbl
    for s in ["train","valid","val","test"]:
        cand=src_dir/s/"images"; lbl=src_dir/s/"labels"
        if cand.exists() and lbl.exists(): return cand,lbl
    return None,None

# очистим train, но оставим val как есть
for p in [TARGET/"images/train", TARGET/"labels/train"]:
    for f in p.glob("*"): f.unlink()

# дедуп по хэшу
def filehash(p, chunk=1<<20):
    h=hashlib.sha256()
    with open(p,'rb') as f:
        while True:
            b=f.read(chunk)
            if not b: break
            h.update(b)
    return h.hexdigest()

seen_hashes=set()
stats = {"kept":0,"badcls":0,"nolabel":0,"full":0,"tiny":0,"train_by_class":{"scratch":0,"dent":0,"rust":0}}

for src_dir in rf_dirs:
    if not (src_dir/"data.yaml").exists():
        print("skip",src_dir,"no data.yaml");
        continue
    data=yaml.safe_load(open(src_dir/"data.yaml"))
    names=data["names"]; id2name = {i:v for i,v in enumerate(names)} if isinstance(names,list) else {int(k):v for k,v in names.items()}

    yaml_splits={}
    for k in ["train","val","valid","test"]:
        if data.get(k): yaml_splits[k]=data[k]

    for k,rel in yaml_splits.items():
        out_split = "val" if k in ("val","valid","test") else "train"
        img_dir,lbl_dir = resolve_imglbl(src_dir, rel, "valid" if k=="val" else k)
        if not img_dir or not lbl_dir:
            print("  ! can't resolve",k,rel);
            continue

        for img in img_dir.rglob("*"):
            if img.suffix.lower() not in IMG_EXT: continue
            lbl = (lbl_dir / img.relative_to(img_dir)).with_suffix(".txt")
            if not lbl.exists(): stats["nolabel"]+=1; continue

            rows=[]
            for line in open(lbl):
                parsed = parse_line(line)
                if not parsed: continue
                cid,(xc,yc,w,h) = parsed
                cname = id2name.get(cid,"")
                t = map_name(cname)
                if t is None: stats["badcls"]+=1; continue
                if is_full(xc,yc,w,h): stats["full"]+=1; continue
                if is_tiny(w,h): stats["tiny"]+=1; continue
                tid = {"scratch":0,"dent":1,"rust":2}[t]
                rows.append((tid,xc,yc,w,h,t))
            if not rows: continue

            # дедуп по содержимому файла (для train)
            if out_split=="train":
                h=filehash(img)
                if h in seen_hashes:
                    continue
                seen_hashes.add(h)

            if out_split=="train":
                out_img = TARGET/"images/train"/img.name
                out_lbl = TARGET/"labels/train"/(img.stem+".txt")
            else:
                out_img = TARGET/"images/val"/img.name
                out_lbl = TARGET/"labels/val"/(img.stem+".txt")

            out_img.parent.mkdir(parents=True, exist_ok=True)
            out_lbl.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(img,out_img)
            with open(out_lbl,"w") as f:
                for tid,xc,yc,w,h,tname in rows:
                    f.write(f"{tid} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")
                    if out_split=="train":
                        stats["train_by_class"][tname]+=1
            stats["kept"]+=1

print("MERGE STATS:", json.dumps(stats, indent=2))


MERGE STATS: {
  "kept": 914,
  "badcls": 5082,
  "nolabel": 0,
  "full": 34823,
  "tiny": 27,
  "train_by_class": {
    "scratch": 723,
    "dent": 370,
    "rust": 80
  }
}


In [ ]:
# простая дубликация файлов редких классов (rust ×3, dent ×1.5)
from collections import defaultdict
import random

train_img_dir = TARGET/"images/train"
train_lbl_dir = TARGET/"labels/train"

def labels_of_class(cls_id):
    out=[]
    for lb in train_lbl_dir.glob("*.txt"):
        txt=lb.read_text().strip().splitlines()
        if any(line.startswith(str(cls_id)+" ") for line in txt):
            out.append(lb)
    return out

dup_plan = {2: 2, 1: 1}  # cls_id: how many extra copies (rust:2 -> х3; dent:1 -> х2)
made=0
for cls_id, k in dup_plan.items():
    lbls = labels_of_class(cls_id)
    for lb in lbls:
        img = train_img_dir/(lb.stem + Path(next(train_img_dir.glob(lb.stem+"*"))).suffix)
        for i in range(k):
            new_img = train_img_dir/(lb.stem+f"_dup{i}.jpg")
            new_lbl = train_lbl_dir/(lb.stem+f"_dup{i}.txt")
            if new_img.exists(): continue
            shutil.copy2(img, new_img)
            shutil.copy2(lb, new_lbl)
            made+=1
print(f"Oversampled files created: {made}")


Oversampled files created: 353


In [ ]:
data_damage_yaml = {
    "path": str(TARGET.resolve()),
    "train": "images/train",
    "val": "images/val",
    "names": ["scratch","dent","rust"]
}
with open("data_damage.yaml","w") as f:
    yaml.safe_dump(data_damage_yaml, f, sort_keys=False, allow_unicode=True)
print(open("data_damage.yaml").read())

print("train imgs:", len(list((TARGET/'images/train').glob('*'))))
print("val imgs:  ", len(list((TARGET/'images/val').glob('*'))))


path: /content/drive/MyDrive/indrive_project/datasets/damage
train: images/train
val: images/val
names:
- scratch
- dent
- rust

train imgs: 1110
val imgs:   157


In [ ]:
%%bash
cat > hyp_det_strong.yaml << 'YAML'
mosaic: 1.0
mixup: 0.15
copy_paste: 0.2
degrees: 0.0
translate: 0.12
scale: 0.6
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
hsv_h: 0.015
hsv_s: 0.70
hsv_v: 0.45
erasing: 0.5
auto_augment: randaugment
crop_fraction: 1.0
YAML
echo "✓ hyp_det_strong.yaml written"


✓ hyp_det_strong.yaml written


In [ ]:
from ultralytics import YOLO
from pathlib import Path
import yaml

with open("hyp_det_strong.yaml", "r") as f:
    hyp_overrides = yaml.safe_load(f) or {}

det = YOLO("yolov8s.pt")
det_res = det.train(
    data="data_damage.yaml",
    imgsz=800,
    epochs=50,
    batch=-1,
    amp=True,
    workers=2,
    close_mosaic=20,
    patience=50,
    project=str(RUNS_DIR),
    name="v8m_det_damage_strong",
    verbose=True,
    **hyp_overrides,           # <-- просто подставляем параметры из YAML
)

DET_PT = Path(det_res.save_dir) / "weights/best.pt"
print("New detector:", DET_PT)

m = det.val(data="data_damage.yaml", imgsz=800, conf=0.25, iou=0.5, plots=True, verbose=True)
print("mAP50:", float(m.box.map50), "mAP50-95:", float(m.box.map))


New https://pypi.org/project/ultralytics/8.3.199 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data_damage.yaml, epochs=50, time=None, patience=50, batch=-1, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=2, project=/content/drive/MyDrive/indrive_project/runs, name=v8m_det_damage_strong3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /content/drive/MyDrive/indrive_project/datasets/damage/labels/train.cache... 1110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1110/1110 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=800 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 6.80G reserved, 6.73G allocated, 1.21G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    11136761       44.77         2.510         15.32         16.31        (1, 3, 800, 800)                    list
    11136761       89.54         2.510         18.94         21.12        (2, 3, 800, 800)                    list
    11136761       179.1         2.510         34.97         43.11        (4, 3, 800, 800)                    list
    11136761       358.1         3.041         68.09          81.1        (8, 3, 800, 800)                    list
    11136761       716.3         5.469         129.2           136       (16, 3, 800, 800)                    list
AutoBatch: WARNING ⚠️ CUDA anomaly detected, using default batch-size 16.
AutoBatch: Using batch-size 16 for CUDA:0 18.72G/14.74G (127%) ✅


train: Scanning /content/drive/MyDrive/indrive_project/datasets/damage/labels/train.cache... 1110 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1110/1110 [00:00<?, ?it/s]

...


50 epochs completed in 0.619 hours.
Optimizer stripped from /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:04<00:00,  1.00it/s]


                   all        157        238      0.371      0.448      0.362      0.142
               scratch        134        204      0.361      0.324      0.256     0.0812
                  dent         23         27      0.423      0.593       0.54      0.292
                  rust          4          7       0.33      0.429      0.291     0.0528
Speed: 0.5ms preprocess, 7.2ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3
New detector: /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3/weights/best.pt
Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/drive/MyDrive/indrive_project/datasets/damage/labels/val.cache... 157 images, 0 backgrounds, 0 corrupt: 100%|██████████| 157/157 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.00it/s]


                   all        157        238      0.408      0.438       0.43      0.172
               scratch        134        204      0.421      0.331      0.328      0.113
                  dent         23         27       0.43      0.556      0.568      0.335
                  rust          4          7      0.375      0.429      0.393     0.0684
Speed: 0.5ms preprocess, 14.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong32
mAP50: 0.42996416638607976 mAP50-95: 0.17220788173578516


In [ ]:
from ultralytics import YOLO
from pathlib import Path
import cv2, numpy as np, math, random, shutil

det = YOLO(str(DET_PT))
out_dir = Path("/content/preview_after_retrain");
if out_dir.exists(): shutil.rmtree(out_dir)
pred = det.predict(source=str(TARGET/"images/val"), imgsz=800, conf=0.25, iou=0.5, save=True, project=str(out_dir), name="val_preds", verbose=False)

saved = list((out_dir/"val_preds").glob("*.jpg")) or list((out_dir/"val_preds").rglob("*.jpg"))
saved = random.sample(saved, min(16, len(saved)))

def make_grid(images):
    ims=[cv2.imread(str(p)) for p in images]
    H=max(i.shape[0] for i in ims); W=max(i.shape[1] for i in ims)
    cols=max(1, int(math.sqrt(len(ims)))); rows=math.ceil(len(ims)/cols)
    canvas=255*np.ones((rows*H, cols*W, 3), np.uint8)
    for i,im in enumerate(ims):
        r,c=divmod(i, cols)
        canvas[r*H:r*H+im.shape[0], c*W:c*W+im.shape[1]] = im
    cv2.imwrite("/content/detector_preview_after_retrain.jpg", canvas)
    return "/content/detector_preview_after_retrain.jpg"

print("Saved grid:", make_grid(saved))


Results saved to /content/preview_after_retrain/val_preds
Saved grid: /content/detector_preview_after_retrain.jpg


In [ ]:
# final steps
import cv2, json, numpy as np
from ultralytics import YOLO

DET_WEIGHTS = str(DET_PT)
CLS_WEIGHTS = str(CLS_PT)
SEG_WEIGHTS = None

det_infer = YOLO(DET_WEIGHTS)
cls_infer = YOLO(CLS_WEIGHTS)
seg_infer = YOLO(SEG_WEIGHTS) if SEG_WEIGHTS else None

CRITICAL_PARTS = {"headlight","front_windshield","rear_window","taillight"}
COLOR = {"scratch": (36,170,255), "dent": (40,220,40), "rust": (50,50,240)}
sev2rank={"minor":1,"moderate":2,"major":3}; rank2sev={v:k for k,v in sev2rank.items()}

def pick_part(box_xyxy, seg_result):
    if seg_result is None or getattr(seg_result, "masks", None) is None or seg_result.masks is None:
        return None
    x1,y1,x2,y2 = [int(v) for v in box_xyxy]
    cx, cy = (x1+x2)//2, (y1+y2)//2
    best = (None, 0.0)
    for m, c in zip(seg_result.masks, seg_result.boxes.cls):
        mask = m.data[0].cpu().numpy().astype("uint8")
        name = seg_result.names[int(c.item())]
        h,w = mask.shape
        if 0 <= cy < h and 0 <= cx < w and mask[cy, cx] > 0:
            sub = mask[y1:y2, x1:x2]
            inter = sub.mean() if sub.size else 0.0
            if inter > best[1]: best = (name, inter)
    return best[0]

def bbox_severity(type_name, area_ratio, part, bw, bh, W, H):
    def sev_by_area(a):
        if a < 0.01: return "minor"
        if a < 0.03: return "moderate"
        return "major"
    if part in CRITICAL_PARTS and area_ratio>0: return "major"
    if type_name=="scratch":
        thin = min(bw,bh)/(max(bw,bh)+1e-6)
        eff = area_ratio*(0.25+0.75*thin)
        if thin<0.15 and area_ratio<0.15: return "moderate" if area_ratio>=0.02 else "minor"
        return sev_by_area(eff)
    return sev_by_area(area_ratio)

def fuse_severity(type_name, sev_bbox, sev_img):
    rb,ri = sev2rank[sev_bbox], sev2rank.get(sev_img,2)
    return rank2sev[min(rb,ri) if type_name=="scratch" else max(rb,ri)]

def integrity_score(area_ratios, parts):
    worst=0.0
    for ar,p in zip(area_ratios,parts):
        worst = max(worst, ar*2.0 if p in CRITICAL_PARTS else ar)
    return 5 if worst==0 else (4 if worst<0.01 else 3 if worst<0.03 else 2 if worst<0.08 else 1)

def draw_box(img, x1,y1,x2,y2, text, color):
    x1,y1,x2,y2 = map(int,[x1,y1,x2,y2])
    cv2.rectangle(img,(x1,y1),(x2,y2),color,2)
    (tw,th),_ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(img,(x1,y1-th-8),(x1+tw+8,y1),(color),-1)
    cv2.putText(img,text,(x1+4,y1-6),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2,cv2.LINE_AA)

def infer_json(image_path: str, angle="front", save_vis=True, out_dir=str(DRIVE_ROOT/"qual_infer")):
    im = cv2.imread(image_path); H,W = im.shape[:2]
    seg = seg_infer.predict(source=im, imgsz=640, conf=0.25, iou=0.5, verbose=False)[0] if seg_infer else None
    det = det_infer.predict(source=im, imgsz=640, conf=0.25, iou=0.5, verbose=False)[0]
    cls = cls_infer.predict(source=im, imgsz=256, verbose=False)[0]
    if getattr(cls,"probs",None) is not None:
        probs = cls.probs.data.tolist(); i = int(np.argmax(probs))
        sev_img, sev_img_conf = cls.names[i], float(probs[i])
    else:
        sev_img, sev_img_conf = "moderate", 0.5

    damages, ars, parts = [], [], []
    vis = im.copy()
    for box, cid, conf in zip(det.boxes.xyxy, det.boxes.cls, det.boxes.conf):
        x1,y1,x2,y2 = [float(v) for v in box.tolist()]
        bw,bh = x2-x1, y2-y1
        ar = max(0.0,(bw*bh)/(W*H))
        tname = det.names[int(cid.item())]
        if tname not in ("scratch","dent","rust"): continue
        part = pick_part([x1,y1,x2,y2], seg)
        sev_b = bbox_severity(tname, ar, part, bw, bh, W, H)
        sev_f = fuse_severity(tname, sev_b, sev_img)
        damages.append({
            "type": tname, "severity_bbox": sev_b, "severity_fused": sev_f,
            "det_confidence": float(conf.item()), "part": part,
            "bbox": [x1,y1,bw,bh], "area_ratio": float(ar)
        })
        ars.append(ar); parts.append(part or "unknown")
        draw_box(vis, x1,y1,x2,y2, f"{tname} | {sev_f}", COLOR.get(tname,(0,200,255)))

    integ = integrity_score(ars, parts)
    out = {
        "angle": angle,
        "severity_image": {"label": sev_img, "confidence": sev_img_conf},
        "integrity": {"score_1to5": integ, "label": "no_issues" if integ==5 else "issues"},
        "damage": damages
    }
    if save_vis:
        outp = Path(out_dir); outp.mkdir(parents=True, exist_ok=True)
        vis_path = str(outp/(Path(image_path).stem+"_viz.jpg"))
        cv2.imwrite(vis_path, vis); out["viz_path"]=vis_path
    return out

test_img = str(next((TARGET/"images/val").glob("*")))
res = infer_json(test_img, angle="auto", save_vis=True)
print(json.dumps(res, ensure_ascii=False, indent=2))
print("viz:", res.get("viz_path"))


{
  "angle": "auto",
  "severity_image": {
    "label": "minor",
    "confidence": 0.5780962705612183
  },
  "integrity": {
    "score_1to5": 5,
    "label": "no_issues"
  },
  "damage": [],
  "viz_path": "/content/drive/MyDrive/indrive_project/qual_infer/car-damages-10-_png.rf.83adda50a448a74f7d78d6ef1a9bae92_viz.jpg"
}
viz: /content/drive/MyDrive/indrive_project/qual_infer/car-damages-10-_png.rf.83adda50a448a74f7d78d6ef1a9bae92_viz.jpg


In [ ]:
# Экспорт ONNX
!yolo export model="{DET_PT}" format=onnx opset=12
!yolo export model="{CLS_PT}" format=onnx opset=12

BUNDLE = DRIVE_ROOT/"bundle_current"
if BUNDLE.exists(): shutil.rmtree(BUNDLE)
(BUNDLE/"weights").mkdir(parents=True, exist_ok=True)

shutil.copy2(DET_PT, BUNDLE/"weights/best_det.pt")
shutil.copy2(CLS_PT, BUNDLE/"weights/best_cls.pt")

# положим infer_json.py
infer_code = Path("infer_json_standalone.py")
infer_code.write_text("""# minimal wrapper around notebook infer (imports same libs)
# NOTE: for production use, prefer the longer infer_json.py from notebook
print('Use the notebook infer_json function or copy it here if you need a standalone file.')
""", encoding="utf-8")
shutil.copy2(infer_code, BUNDLE/"infer_json.py")

(Path(BUNDLE/"requirements.txt")).write_text(
"ultralytics==8.3.50\nopencv-python-headless==4.10.0.84\ntorch>=2.2.0\nnumpy>=1.24\n",
encoding="utf-8"
)

# Add data.yaml to the bundle
shutil.copy2("data_damage.yaml", BUNDLE/"data_damage.yaml")


ZIP_OUT = BUNDLE_DIR/"indrive_damage_bundle_current.zip"
if ZIP_OUT.exists(): ZIP_OUT.unlink()
with zipfile.ZipFile(ZIP_OUT, "w", zipfile.ZIP_DEFLATED) as z:
    for p in BUNDLE.rglob("*"):
        z.write(p, p.relative_to(BUNDLE))
print("✓ Saved bundle to:", ZIP_OUT)

Ultralytics 8.3.50 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/indrive_project/runs/v8m_det_damage_strong3/weights/best.pt' with input shape (1, 3, 800, 800) BCHW and output shape(s) (1, 7, 13125) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 214.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 230.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 198.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 159.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 214.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 8.3s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime']
requirements: ⚠️ R

In [ ]:
import shutil, zipfile, os
from pathlib import Path

SAVE_DIR = Path("/content/indrive_models_export")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

weights = [
    "/content/drive/MyDrive/indrive_project/runs/v8s_det_damage/weights/best.pt",
    "/content/drive/MyDrive/indrive_project/runs/v8s_cls_severity/weights/best.pt",
]
for w in weights:
    if Path(w).exists():
        shutil.copy(w, SAVE_DIR)

for f in ["data_damage.yaml", "data_severity.yaml"]:
    if Path(f).exists():
        shutil.copy(f, SAVE_DIR)

# zip
zip_path = "/content/indrive_models_export.zip"
shutil.make_archive(zip_path.replace(".zip",""), 'zip', SAVE_DIR)
print("✓ Exported:", zip_path)


✓ Exported: /content/indrive_models_export.zip
